In [35]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import os


pd.set_option('display.max_columns', None)

## Accidents
### Read in Accidents DF

In [36]:
df = gpd.read_file('/Users/Dario/Documents/GitHub/GIS_KARTO/data/accidents.json')

keep_cols = ['AccidentType_de', 'AccidentSeverityCategory_de', 'AccidentInvolvingPedestrian', 'AccidentInvolvingBicycle', 'RoadType_de', 'AccidentLocation_CHLV95_E', 'AccidentLocation_CHLV95_N', 'CantonCode', 'AccidentYear', 'AccidentMonth', 'geometry' ]

In [37]:
gdf = df.drop(columns=[col for col in df.columns if col not in keep_cols])

In [38]:
# Typechange of DF

gdf['AccidentInvolvingBicycle'] = gdf['AccidentInvolvingBicycle'].str.lower().map({'true': True, 'false': False})
gdf['AccidentInvolvingPedestrian'] = gdf['AccidentInvolvingPedestrian'].str.lower().map({'true': True, 'false': False})
gdf.AccidentYear = gdf.AccidentYear.astype(int)
gdf.AccidentMonth = gdf.AccidentMonth.astype(int)
gdf.AccidentLocation_CHLV95_E = gdf.AccidentLocation_CHLV95_E.astype(int)
gdf.AccidentLocation_CHLV95_N = gdf.AccidentLocation_CHLV95_N.astype(int)



In [39]:
# Filtering DF
#gdf = gdf[gdf['AccidentYear'] <= 2018]
gdf = gdf[(gdf['AccidentMonth'] <= 9) & (gdf['AccidentYear'] <= 2018)]
gdf = gdf[gdf['AccidentInvolvingBicycle']]

In [41]:
gdf = gdf.set_crs(gdf.crs)

geometry = gpd.points_from_xy(gdf['AccidentLocation_CHLV95_E'], gdf['AccidentLocation_CHLV95_N'], crs='epsg:2056')

gdf.geometry = geometry

In [45]:
#gdf.to_file('../../export/accidents/accidents.geojson', index=False)

## Master Table

In [7]:
# Read MasterTable 
master_table = gpd.read_file('../../export/master.geojson')
#master_table = pd.read_csv('../../export/master.geojson', sep=',')
#master_table['geometry'] = master_table['geometry'].apply(wkt.loads)
#master = gpd.GeoDataFrame(master_table, geometry='geometry')

In [16]:
master_table.crs

<Derived Projected CRS: EPSG:2056>
Name: CH1903+ / LV95
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Liechtenstein; Switzerland.
- bounds: (5.96, 45.82, 10.49, 47.81)
Coordinate Operation:
- name: Swiss Oblique Mercator 1995
- method: Hotine Oblique Mercator (variant B)
Datum: CH1903+
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

## Compare Accidents with Master Table

In [17]:
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon

In [18]:
gdf = gdf.set_crs(gdf.crs)

geometry = gpd.points_from_xy(gdf['AccidentLocation_CHLV95_E'], gdf['AccidentLocation_CHLV95_N'], crs='epsg:2056')

gdf.geometry = geometry

In [19]:
# Load the GeoDataFrames
polygons_gdf = master_table #resulate df
points_gdf = gdf #acc df

In [20]:
# Subset for faster computation
small_poly = polygons_gdf
small_point = points_gdf

In [21]:
# Perform the spatial join
#joined_gdf = sjoin(small_point, small_poly, op='within')

# Count the number of points per polygon and create a new column in the polygons GeoDataFrame
#acc_count = joined_gdf.groupby('index_right').size().reset_index(name='Acc_Count')
#small_poly = small_poly.merge(acc_count, on='index_right', how='left')


In [22]:
small_poly['Acc_Count'] = 0

In [23]:
print(small_poly.crs, small_point.crs)

epsg:2056 epsg:2056


In [26]:
import geopandas as gpd

# perform spatial join to get points within polygons
joined = gpd.sjoin(small_point, small_poly, op='within')

# groupby polygon index and count number of points for each polygon
counts = joined.groupby('index_right').size()

# update small_poly with the point counts
small_poly['Acc_Count'] = counts


/Users/Dario/opt/anaconda3/envs/GTA22/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3472: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [34]:
small_poly

,GMDNR,GMDNAME_x,Stimmberechtigte,Abgegebene Stimmen,Beteiligung in %,Gültige Stimmzettel,Ja,Nein,Ja in %,GMDNAME_y,BZNR,KTNR,E_CNTR,N_CNTR,geometry,Acc_Count
0,1,Aeugst am Albis,1402,664,47.36,663,481,182,72.55,Aeugst am Albis,101,1,2679300,1235700,"POLYGON ((2678219.000 1235219.000, 2678439.000...",2.0
1,2,Affoltern am Albis,7186,2880,40.08,2850,2079,771,72.95,Affoltern am Albis,101,1,2676800,1236800,"POLYGON ((2678219.000 1235219.000, 2677940.000...",36.0
2,3,Bonstetten,3656,1701,46.53,1691,1251,440,73.98,Bonstetten,101,1,2677800,1241000,"POLYGON ((2675803.000 1241039.000, 2675748.000...",11.0
3,4,Hausen am Albis,2505,1145,45.71,1132,802,330,70.84999999999999,Hausen am Albis,101,1,2682900,1233100,"POLYGON ((2686324.000 1230388.000, 2686024.000...",11.0
4,5,Hedingen,2515,1183,47.04,1178,876,302,74.36,Hedingen,101,1,2676400,1239000,"POLYGON ((2675803.000 1241039.000, 2676538.000...",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2131,6806,Vendlincourt,472,139,29.45,134,97,37,72.39,Vendlincourt,2603,26,2578200,1255600,"POLYGON ((2580412.000 1257077.000, 2580367.000...",1.0
2132,6807,Basse-Allaine,951,264,27.76,260,191,69,73.45999999999999,Basse-Allaine,2603,26,2569300,1258900,"POLYGON ((2567009.521 1256868.290, 2566957.590...",NaN
2133,6808,Clos du Doubs,1096,338,30.84,329,270,59,82.06999999999999,Clos du Doubs,2603,26,2579100,1246300,"POLYGON ((2569410.484 1246416.863, 2571130.000...",1.0
2134,6809,Haute-Ajoie,983,318,32.35,314,224,90,71.34,Haute-Ajoie,2603,26,2567000,1249100,"POLYGON ((2559718.024 1248149.921, 2560200.000...",NaN


In [148]:
for idx, point in small_point.geometry.items():
    # loop through the small_poly GeoDataFrame with index and polygon
    for poly_idx, poly in small_poly.geometry.items():
        # check if the point is within the polygon
        if point.within(poly):
            # increment the count of points for the polygon
            print(point, poly_idx)
            small_poly.at[poly_idx, 'Acc_Count'] += 1
        else:
            pass

In [117]:
joined = gpd.sjoin(small_poly, small_point, op='contains')

# group the resulting GeoDataFrame by the polygons and count the number of points
counts = joined.groupby('GMDNR').size().reset_index(name='Acc_Count')

# merge the counts back to the small_poly GeoDataFrame
small_poly = small_poly.merge(counts, on='GMDNR', how='left')

/Users/Dario/opt/anaconda3/envs/GTA22/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3472: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
